In [2]:
from requests import get
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

In [19]:
API_data = pd.read_csv('C:/Users/harini/Desktop/DMDD/APIdata.csv')
url_list = API_data[API_data.links.notnull()]['links']
print(url_list)

0           https://twitter.com/RepAbraham
1             https://twitter.com/RepAdams
2      https://twitter.com/Robert_Aderholt
3       https://twitter.com/reppeteaguilar
4         https://twitter.com/reprickallen
6        https://twitter.com/MarkAmodeiNV2
7         https://twitter.com/RepArrington
8        https://twitter.com/RepBrianBabin
9          https://twitter.com/repdonbacon
10        https://twitter.com/RepBalderson
11         https://twitter.com/RepJimBanks
12      https://twitter.com/RepLouBarletta
13         https://twitter.com/RepAndyBarr
14         https://twitter.com/RepBarragan
15        https://twitter.com/RepJoeBarton
16        https://twitter.com/RepKarenBass
17           https://twitter.com/RepBeatty
18          https://twitter.com/RepBecerra
19             https://twitter.com/RepBera
20      https://twitter.com/RepJackBergman
21         https://twitter.com/RepDonBeyer
22      https://twitter.com/RepAndyBiggsAZ
23     https://twitter.com/RepGusBilirakis
24       ht

In [55]:
#creating lists for storing data
account_name = []
account = []
user_name = []
tweet = []
time = []
retweet = []
likes = []
comments = []
hashtags = []
tags = []
replies = []
print(len(account_name),len(account),len(user_name),len(tweet),len(time),len(retweet),len(likes),len(hashtags),len(tags),len(replies))

0 0 0 0 0 0 0 0 0 0


In [56]:
for url in url_list:
    response  = get(url)
    html_soup = bs(response.text,'html.parser')
    id_check = html_soup.find(id ="stream-items-id")
    if(id_check != None):
        twitter_container = id_check.find_all(class_ ="js-stream-item")
        if(twitter_container != None):
            for i in range(0,len(twitter_container)):
                container = twitter_container[i]
                if (container != None):
                    tweet_container = container.find("p", class_ = "js-tweet-text")
                    account_name.append(container.find("strong", class_="fullname").text)
                    user_name.append(container.find("span", class_="username").text)
                    account.append(container.find("span", class_="UserBadges").text)
                    if(tweet_container != None):
                        pic = tweet_container.find("a", class_="u-hidden")
                        if (pic != None):
                            tweet.append(tweet_container.text.replace(pic.text," "))
                        else:
                            tweet.append(tweet_container.text)
                        has = tweet_container.find_all("a", class_ = "twitter-hashtag")
                        h = []
                        if(has != None):
                            for j in range(0,len(has)):
                                h.append(has[j].text)
                            hashtags.append(h)
                        else:
                            hashtags.append(None)
                        rep = tweet_container.find("a", class_= "twitter-atreply")
                        r = []
                        if(rep != None):
                            for k in range(0,len(rep)):
                                r.append(rep.text)
                            tags.append(r)
                        else:
                            tags.append(None)
                    else:
                        tweet.append(None)
                        hashtags.append(None)
                        tags.append(None)
                    retweet_container = container.find("div", class_ = "ProfileTweet-action--retweet")
                    if(retweet_container != None):
                        retweet.append(retweet_container.find("span", class_= "ProfileTweet-actionCountForPresentation").text)
                    else:
                        retweet.append(None)
                    favorite_container = container.find("div", class_ ="ProfileTweet-action--favorite")
                    if(favorite_container != None):
                        likes.append(favorite_container.find("span", class_ = "ProfileTweet-actionCountForPresentation").text)
                    else:
                        likes.append(None)
                    reply_container = container.find("div", class_="ProfileTweet-action--reply")
                    if(reply_container != None):
                        replies.append(reply_container.find("span", class_ = "ProfileTweet-actionCountForPresentation").text)
                    else:
                        replies.append(None)
                    t = container.find("a", class_ = "tweet-timestamp")
                    if(t != None):
                        time.append(container.find("a", class_ = "tweet-timestamp").text)
                    else:
                        time.append("0 sec")

In [57]:
print(len(account_name),len(account),len(user_name),len(tweet),len(time),len(retweet),len(likes),len(hashtags),len(tags),len(replies))

8805 8805 8805 8805 8805 8805 8805 8805 8805 8805


In [58]:
#converting lists to dataframe
twitter_data = pd.DataFrame(np.column_stack([account_name,user_name,account,tweet,hashtags,tags,replies,retweet,likes,time]),
                           columns=['account_name','user_name','account_type','tweets','hashtags','tags','replies','retweets','likes','time'])

In [80]:
twitter_data.head()

,account_name,user_name,account_type,tweets,hashtags,tags,replies,retweets,likes,time
0,Rep. Ralph Abraham,@RepAbraham,Verified account,"This week, I’m proud to support National Schoo...",[#LA05],None,7,3,16,Jan 23
1,Rep. Ralph Abraham,@RepAbraham,Verified account,Today we celebrate the legacy of Dr. Martin Lu...,[],None,4,1,21,Jan 21
2,Rep. Ralph Abraham,@RepAbraham,Verified account,"As a doctor, I've spent my career protecting l...",[],None,11,8,41,Jan 18
3,Rep. Ralph Abraham,@RepAbraham,Verified account,Great to meet some seniors from Opelousas Cath...,[],None,4,4,26,Jan 15
4,Steve Scalise,@SteveScalise,Verified account,Glad to see the steel slats are deterring ille...,"[#BuildTheWall, #WallsWork]",{@realDonaldTrump},,,,Jan 10


In [77]:
twitter_data['hashtags'][0] == None

False

In [70]:
twitter_data['tags'][4] = set(twitter_data['tags'][4])

In [79]:
for m in range(0,len(twitter_data['tags'])):
    if (twitter_data['tags'][m] != None):
        twitter_data['tags'][m] = set(twitter_data['tags'][m])